In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os

In [3]:
# Load the dataset
path = kagglehub.dataset_download("lakshmi25npathi/online-retail-dataset")

dataset_path = r"C:\Users\victo\.cache\kagglehub\datasets\lakshmi25npathi\online-retail-dataset\versions\1"
files = os.listdir(dataset_path)

file_path = r"C:\Users\victo\.cache\kagglehub\datasets\lakshmi25npathi\online-retail-dataset\versions\1\online_retail_II.xlsx"
df = pd.read_excel(file_path)

# Clean the data
df['Description'] = df.groupby('StockCode')['Description'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)
df = df.dropna(subset=['Customer ID'])
df.drop_duplicates(inplace=True)

# Apply Features to the dataset
df["Revenue"] = df["Quantity"] * df["Price"] 

df["TotalRevenue"] = df.groupby("Customer ID")["Revenue"].transform("sum")

df["Tier"] = df["TotalRevenue"].apply(lambda x: "Low" if x < 724.26 else ("Medium" if x < 1187.08 else "High" ))

is_christmas_item = lambda x: True if "CHRISTMAS" in str(x) else False
df["ChristmasItem"] = df["Description"].apply(is_christmas_item) 

df["NumberOfPurchases"] = df.groupby("Customer ID")["Invoice"].transform("nunique")

customer_categori = lambda x: "One Time" if x <= 1 else ("Loyal" if x > 5 else "Occasional")
df["CustomerCategori"]=df["NumberOfPurchases"].apply(customer_categori)

df_grouped_invoices = df.groupby("Invoice")["Invoice"].transform("count")
multi_invoice = lambda x: True if x > 1 else False
df["MultiInvoice"] = df_grouped_invoices.apply(multi_invoice)

In [7]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,TotalRevenue,Tier,ChristmasItem,NumberOfPurchases,CustomerCategori,MultiInvoice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,1187.08,High,True,7,Loyal,True
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,1187.08,High,False,7,Loyal,True
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,1187.08,High,False,7,Loyal,True
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,1187.08,High,False,7,Loyal,True
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,1187.08,High,False,7,Loyal,True
